# Tree Mapping with SAM2

This notebook demonstrates **tree crown segmentation** using the Segment Anything Model 2 (SAM2).

**Based on:** [segment-geospatial tree mapping example](https://samgeo.gishub.org/examples/tree_mapping)

**What you'll learn:**
- Download satellite imagery as GeoTIFF
- Use SAM2 for automatic tree crown detection
- Extract vector polygons and centroid points
- Visualize results with purple polygons and yellow dots

## 1. Install Dependencies (Optional)

Uncomment the lines below if you need to install the required packages:

In [1]:
# Uncomment to install if needed
%pip install segment-geospatial[samgeo2]
%pip install leafmap geopandas

  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install gdal

Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

In [2]:
import leafmap
from samgeo import SamGeo2
import geopandas as gpd
import os

print("✓ Libraries imported successfully")

Please install required dependencies as:
	pip install segment-geospatial[samgeo3]
✓ Libraries imported successfully


## 3. Create Interactive Map

Create a map centered on the study area (Brazil example) with satellite basemap.

**Instructions:**
- Pan/zoom to explore the area
- Use the draw tool (rectangle/polygon) to define your area of interest (optional)

In [3]:
m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[-22.17615, -51.253043], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## 4. Download Satellite Image as GeoTIFF

Download the satellite imagery for the selected area.

In [4]:
import os
from osgeo import gdal

# Configure GDAL for better compatibility
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_TIMEOUT', '60')
gdal.UseExceptions()

# Define image and bbox
image = "Image.tif"
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.2565, -22.1777, -51.2512, -22.175]

print(f"Attempting to download satellite imagery...")
print(f"Bounding box: {bbox}")
print(f"Output file: {os.path.abspath(image)}")

try:
    # Try using leafmap's tile download function
    leafmap.map_tiles_to_geotiff(
        output=image, 
        bbox=bbox, 
        zoom=19, 
        source="Satellite", 
        overwrite=True
    )
    
    print(f"✓ Image downloaded successfully: {image}")
    print(f"File size: {os.path.getsize(image) / (1024*1024):.2f} MB")
    
except Exception as e:
    print(f"✗ Tile download failed: {e}")
    print("\n" + "="*60)
    print("WORKAROUND: Using samgeo's built-in test image")
    print("="*60)
    
    # Alternative: Download a test image from a simple source
    import requests
    test_url = "https://github.com/opengeos/leafmap/raw/master/examples/data/elevation.tif"
    
    print(f"\nDownloading test image from: {test_url}")
    response = requests.get(test_url, allow_redirects=True, timeout=30)
    response.raise_for_status()
    
    with open(image, 'wb') as f:
        f.write(response.content)
    
    print(f"✓ Test image downloaded: {image}")
    print(f"File size: {os.path.getsize(image) / (1024*1024):.2f} MB")
    print("\n⚠ Note: This is a test image. For best results, use your own imagery.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 758,

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Attempting to download satellite imagery...
Bounding box: [-51.2565, -22.1777, -51.2512, -22.175]
Output file: c:\Users\LEGION\Documents\GitHub\segment-geospatial\Image.tif



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 758,

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Downloaded image 1/45
Downloaded image 2/45
Downloaded image 3/45
Downloaded image 4/45
Downloaded image 5/45
Downloaded image 6/45
Downloaded image 7/45
Downloaded image 8/45
Downloaded image 9/45
Downloaded image 10/45
Downloaded image 11/45
Downloaded image 12/45
Downloaded image 13/45
Downloaded image 14/45
Downloaded image 15/45
Downloaded image 16/45
Downloaded image 17/45
Downloaded image 18/45
Downloaded image 19/45
Downloaded image 20/45
Downloaded image 21/45
Downloaded image 22/45
Downloaded image 23/45
Downloaded image 24/45
Downloaded image 25/45
Downloaded image 26/45
Downloaded image 27/45
Downloaded image 28/45
Downloaded image 29/45
Downloaded image 30/45
Downloaded image 31/45
Downloaded image 32/45
Downloaded image 33/45
Downloaded image 34/45
Downloaded image 35/45
Downloaded image 36/45
Downloaded image 37/45
Downloaded image 38/45
Downloaded image 39/45
Downloaded image 40/45
Downloaded image 41/45
Downloaded image 42/45
Downloaded image 43/45
Downloaded image 44/

HTTPError: 404 Client Error: Not Found for url: https://github.com/opengeos/leafmap/raw/master/examples/data/elevation.tif

### Display the Downloaded Image

In [5]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

[2025-12-02 12:14:00,160] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask_restx\api.py", line 404, in wrapper
    resp = resource(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\views.py", line 110, in view
    return current_app.ensure_sync(self.dispatch_request)(**kwargs)  # type: ignore

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:54623/api/metadata?&filename=c%3A%5CUsers%5CLEGION%5CDocuments%5CGitHub%5Csegment-geospatial%5CImage.tif

## 5. Initialize SAM2 Model

Load the SAM2 model and set the image for segmentation.

**Note:** First time running this will download model weights (~600MB). This may take a few minutes.

In [8]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,      # We will provide bounding boxes as prompts
    # device="cuda"       # Uncomment if you have GPU
    # device="cpu"        # Explicitly force CPU if needed
)

# Load the image into SAM
sam.set_image(image)

print("✓ SAM2 model initialized and image loaded")

sam2_hiera_large.pt:   0%|          | 0.00/898M [00:00<?, ?B/s]

✓ SAM2 model initialized and image loaded


## 6. Load Pre-made Tree Bounding Boxes

Load bounding boxes from a GeoJSON file that contains tree locations from the example dataset.

In [17]:
geojson = "https://github.com/opengeos/datasets/releases/download/samgeo/tree_boxes.geojson"

# Create a new map for visualization
m2 = leafmap.Map()

# Add raster using COG (Cloud Optimized GeoTIFF) approach to avoid localtileserver issues
try:
    m2.add_raster(image, layer_name="Image")
except Exception as e:
    print(f"Note: Could not add raster preview: {e}")
    print("Continuing with vector overlay...")

# Style for bounding boxes (yellow outlines)
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}

m2.add_vector(
    geojson,
    style=style,
    zoom_to_layer=True,
    layer_name="Bounding boxes",
    info_mode=None,
)

print("✓ Bounding boxes loaded from:", geojson)
m2

[2025-12-02 12:08:04,558] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\osgeo\data\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rul

Note: Could not add raster preview: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:58580/api/metadata?&filename=c%3A%5CUsers%5CLEGION%5CDocuments%5CGitHub%5Csegment-geospatial%5CImage.tif
Continuing with vector overlay...
✓ Bounding boxes loaded from: https://github.com/opengeos/datasets/releases/download/samgeo/tree_boxes.geojson
✓ Bounding boxes loaded from: https://github.com/opengeos/datasets/releases/download/samgeo/tree_boxes.geojson


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## 7. Segment Trees Using Bounding Boxes

Run SAM2 prediction using the bounding boxes as prompts. SAM will segment trees within each box.

In [13]:
output_masks = "mask2.tif"

sam.predict(
    boxes=geojson,
    point_crs="EPSG:4326",
    output=output_masks,
    dtype="uint8",
)

print(f"✓ Segmentation complete: {output_masks}")

Some coordinates are out of the image boundary.
✓ Segmentation complete: mask2.tif
✓ Segmentation complete: mask2.tif


### Visualize Raw Segmentation Masks

In [14]:
m2.add_raster(output_masks, nodata=0, opacity=0.5, layer_name="Tree masks")
m2

[2025-12-02 12:06:30,182] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\osgeo\data\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rul

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:58580/api/metadata?&filename=c%3A%5CUsers%5CLEGION%5CDocuments%5CGitHub%5Csegment-geospatial%5Cmask2.tif

## 8. Clean Masks and Extract Vector Polygons

Use region grouping to clean up the masks and convert them to vector polygons. This step:
- Filters out very small regions (noise)
- Creates individual labels for each tree crown
- Exports results as both raster and vector formats

In [18]:
out_image = "tree_masks.tif"
out_vector = "tree_vector.geojson"

array, gdf = sam.region_groups(
    output_masks,
    min_size=200,          # Filter very small regions (adjust as needed)
    out_vector=out_vector,
    out_image=out_image,
)

print(f"✓ Cleaned masks saved to: {out_image}")
print(f"✓ Vector polygons saved to: {out_vector}")
print(f"\nFound {len(gdf)} tree crowns")

# Display first few rows
gdf.head()

✓ Cleaned masks saved to: tree_masks.tif
✓ Vector polygons saved to: tree_vector.geojson

Found 73 tree crowns


,geometry,label,area,area_bbox,area_convex,area_filled,axis_major_length,axis_minor_length,eccentricity,equivalent_diameter_area,extent,orientation,perimeter,solidity,elongation
16,"POLYGON ((-5705817.622 -2532549.081, -5705817....",1,14045.0,24500.0,15244.0,14045.0,207.194300,90.962177,0.898478,133.726024,0.573265,0.467861,545.546248,0.921346,2.277807
1,"POLYGON ((-5705660.868 -2532549.081, -5705660....",2,3200.0,3816.0,3300.0,3200.0,111.383982,37.388288,0.941980,63.830765,0.838574,1.537083,257.396970,0.969697,2.979114
0,"POLYGON ((-5705576.668 -2532549.081, -5705576....",3,1840.0,2412.0,1897.0,1840.0,68.039572,36.476242,0.844152,48.402074,0.762852,1.522526,179.154329,0.969953,1.865312
18,"POLYGON ((-5705527.104 -2532549.081, -5705527....",4,27884.0,44128.0,32369.0,27884.0,215.310617,184.947541,0.512009,188.422428,0.631889,1.190028,1001.003138,0.861442,1.164171
2,"POLYGON ((-5705415.435 -2532549.081, -5705415....",5,4220.0,6420.0,4648.0,4220.0,105.167695,53.861109,0.858899,73.301234,0.657321,1.308943,286.551299,0.907917,1.952572


## 9. Create Centroid Points (Yellow Dots)

Extract the centroid (center point) of each tree crown polygon. These will be displayed as yellow dots.

In [19]:
poly_gdf = gpd.read_file(out_vector)
pt_gdf = poly_gdf.copy()
pt_gdf["geometry"] = pt_gdf.geometry.centroid

pt_points = "tree_points.geojson"
pt_gdf.to_file(pt_points, driver="GeoJSON")

print(f"✓ Centroid points saved to: {pt_points}")
print(f"Total points: {len(pt_gdf)}")

✓ Centroid points saved to: tree_points.geojson
Total points: 73


## 10. Final Visualization: Purple Polygons + Yellow Centroids

Create the final map showing:
- **Colored tree masks** (multi-colored crowns)
- **Purple polygon outlines** (tree crown boundaries)
- **Yellow centroid points** (tree centers)

This recreates the visualization from the tutorial!

In [20]:
m_final = leafmap.Map()

# Try to add rasters (may fail due to localtileserver issues)
try:
    m_final.add_raster(image, layer_name="Image")
except Exception as e:
    print(f"Note: Image preview skipped: {e}")

try:
    m_final.add_raster(
        out_image,
        colormap="tab20",
        nodata=0,
        opacity=0.7,
        layer_name="Tree masks",
    )
except Exception as e:
    print(f"Note: Mask preview skipped: {e}")

# Purple polygon outlines - these should work
poly_style = {
    "color": "#7c4185",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.3,
}
m_final.add_vector(out_vector, style=poly_style, zoom_to_layer=True, layer_name="Tree polygons")

# Yellow centroid points
point_style = {
    "color": "#ffff00",
    "fillColor": "#ffff00",
    "radius": 5,
    "weight": 2,
}
m_final.add_vector(pt_points, style=point_style, layer_name="Tree centroids")

m_final.add_layer_manager()
print("✓ Final visualization ready!")
print(f"  - {len(poly_gdf)} tree crown polygons (purple)")
print(f"  - {len(pt_gdf)} centroid points (yellow)")
m_final

[2025-12-02 12:09:00,851] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\osgeo\data\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rul

Note: Image preview skipped: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:58580/api/metadata?&filename=c%3A%5CUsers%5CLEGION%5CDocuments%5CGitHub%5Csegment-geospatial%5CImage.tif


[2025-12-02 12:09:03,081] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\osgeo\data\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rul

Note: Mask preview skipped: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:58580/api/metadata?&filename=c%3A%5CUsers%5CLEGION%5CDocuments%5CGitHub%5Csegment-geospatial%5Ctree_masks.tif
✓ Final visualization ready!
  - 73 tree crown polygons (purple)
  - 73 centroid points (yellow)
✓ Final visualization ready!
  - 73 tree crown polygons (purple)
  - 73 centroid points (yellow)


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## 11. Summary Statistics

Print summary information about the detected trees.

In [ ]:
print("=" * 60)
print("TREE MAPPING SUMMARY")
print("=" * 60)
print(f"Total tree crowns detected: {len(poly_gdf)}")

# Calculate area statistics
if 'area' in poly_gdf.columns:
    print(f"\nArea statistics (square meters):")
    print(f"  Mean area: {poly_gdf['area'].mean():.2f}")
    print(f"  Min area: {poly_gdf['area'].min():.2f}")
    print(f"  Max area: {poly_gdf['area'].max():.2f}")
else:
    # Calculate area from geometry
    areas = poly_gdf.geometry.area
    print(f"\nArea statistics:")
    print(f"  Mean area: {areas.mean():.6f}")
    print(f"  Min area: {areas.min():.6f}")
    print(f"  Max area: {areas.max():.6f}")

print("\n✓ All outputs saved successfully!")
print("=" * 60)

# List output files
print("\nOutput Files:")
print(f"  • {image} - Original satellite image")
print(f"  • {output_masks} - Raw segmentation mask")
print(f"  • {out_image} - Cleaned tree masks (raster)")
print(f"  • {out_vector} - Tree crown polygons (GeoJSON)")
print(f"  • {pt_points} - Tree centroid points (GeoJSON)")

TREE MAPPING SUMMARY
Total tree crowns detected: 73

Area statistics (square meters):
  Mean area: 5931.27
  Min area: 231.00
  Max area: 59035.00

✓ All outputs saved successfully!

Output Files:
  • Image.tif - Original satellite image
  • mask2.tif - Raw segmentation mask
  • tree_masks.tif - Cleaned tree masks (raster)
  • tree_vector.geojson - Tree crown polygons (GeoJSON)
  • tree_points.geojson - Tree centroid points (GeoJSON)


[2025-12-02 12:09:32,736] ERROR in app: Exception on /api/metadata [GET]
Traceback (most recent call last):
  File "rasterio\\crs.pyx", line 592, in rasterio.crs.CRS.from_epsg
  File "rasterio\\_err.pyx", line 289, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: PROJ: proj_create_from_database: c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\osgeo\data\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LEGION\Desktop\PLant identification\.conda\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rul